<a href="https://colab.research.google.com/github/satyamnewale/basic-NLP/blob/main/nn-and-optmise_module-CancerDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#create a neural network

In [17]:
#imports
import torch
import torch.nn as nn

class Model(nn.Module):

  ##architecture
  def __init__(self, num_features):
    super().__init__()
    # self.linear1 = nn.Linear(num_features, 3) # (I/P no., O/P no.)
    # self.relu = nn.ReLU()
    # self.linear2 = nn.Linear(3,1)
    # self.sigmoid = nn.Sigmoid()

    # sequential container
    self.network = nn.Sequential(
         nn.Linear(num_features, 3), # (I/P no., O/P no.)
         nn.ReLU(),
         nn.Linear(3,1),
         nn.Sigmoid(),
    )

  ##forward pass

  def forward(self, features):
  #   out = self.linear1(features)
  #   out = self.relu(out)
  #   out = self.linear2(out)
  #   out = self.sigmoid(out) #y_pred

    out = self.network(features)
    return out

In [18]:
#create dataset
features = torch.rand(10,5)

#create model
model = Model(features.shape[1])

#call model for forward pass
model(features) # forward call (magic function)

model.network[2].weight

Parameter containing:
tensor([[ 0.0756, -0.0280, -0.3284]], requires_grad=True)

In [19]:
!pip install torchinfo

In [20]:
from torchinfo import summary

summary(model, input_size= (10,5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 3]                   18
│    └─ReLU: 2-2                         [10, 3]                   --
│    └─Linear: 2-3                       [10, 1]                   4
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

---

In [21]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [22]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [23]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)
df.head()


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [24]:
df.shape

(569, 31)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=0.2)

In [26]:
#scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
X_train

array([[-1.15135509, -1.0211642 , -1.15350819, ..., -1.33157355,
         1.10129218, -0.18227611],
       [-0.55626954, -0.324774  , -0.58510266, ..., -0.29135159,
         0.93328557, -0.81908758],
       [ 1.12980621, -0.14609493,  1.13282718, ...,  0.26488964,
         0.21843393, -0.20097253],
       ...,
       [-0.53643335,  1.72316299, -0.47273389, ...,  0.46536389,
         0.53797591,  2.10095365],
       [ 1.40467906, -0.77834394,  1.37889018, ...,  0.94111621,
        -0.73360351, -1.18451766],
       [ 1.2544908 , -0.50803458,  1.17793873, ...,  0.78702033,
         0.00924923,  0.06474329]])

In [8]:
#label encoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
y_train

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,

In [9]:
#numpy to pytorch tensors
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train.astype(np.float64))
y_test_tensor = torch.from_numpy(y_test.astype(np.float64))

In [10]:
X_train_tensor

tensor([[-1.1514, -1.0212, -1.1535,  ..., -1.3316,  1.1013, -0.1823],
        [-0.5563, -0.3248, -0.5851,  ..., -0.2914,  0.9333, -0.8191],
        [ 1.1298, -0.1461,  1.1328,  ...,  0.2649,  0.2184, -0.2010],
        ...,
        [-0.5364,  1.7232, -0.4727,  ...,  0.4654,  0.5380,  2.1010],
        [ 1.4047, -0.7783,  1.3789,  ...,  0.9411, -0.7336, -1.1845],
        [ 1.2545, -0.5080,  1.1779,  ...,  0.7870,  0.0092,  0.0647]],
       dtype=torch.float64)

In [56]:
loss_function = nn.BCELoss()

In [53]:
# #defining the model

# class MySimpleNN():

#   def __init__(self, X):
#     self.weights = torch.rand(X.shape[1],1, dtype = torch.float64, requires_grad=True)
#     self.bias = torch.zeros(1, dtype = torch.float64, requires_grad=True)

#   def forward(self, X):
#     z = torch.matmul(X, self.weights) + self.bias
#     y_pred = torch.sigmoid(z)
#     return y_pred

#   def loss_function(self, y_pred, y):
#     ## clamps prediction to avoid log(0) and log(1)
#     epsilon = 1e-7
#     y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
#     ## calculate loss
#     loss = (-(y*torch.log(y_pred) + (1-y)*torch.log(1-y_pred))).mean()
#     return loss

class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, X):
    out = self.linear(X)
    out = self.sigmoid(out)
    return out

  # def loss_function(self, y_pred, y):
  #   ## clamps prediction to avoid log(0) and log(1)
  #   epsilon = 1e-7
  #   y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
  #   ## calculate loss
  #   loss = (-(y*torch.log(y_pred) + (1-y)*torch.log(1-y_pred))).mean()
  #   return loss

In [46]:
#important parameters

learning_rate = 0.1
epochs = 2500

In [29]:
X_train_tensor.shape[1]

30

In [68]:
#training pipeline
## create model
model = MySimpleNN(X_train_tensor.shape[1])
model = model.double() # Convert model parameters to double

#define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## loop
for epoch in range(epochs):
  # X_train_tensor is already float64 (Double)
  # y_train_tensor is already float64 (Double), reshape it
  y_train_tensor_reshaped = y_train_tensor.unsqueeze(1)

  ## forward pass
  y_pred = model.forward(X_train_tensor) # Use X_train_tensor directly

  ## loss
  loss = loss_function(y_pred, y_train_tensor_reshaped)

  ## zero gradient
  optimizer.zero_grad()

  ## backward pass
  loss.backward()

  #now update weights and bias
  # with torch.no_grad():
  #   model.linear.weight -= learning_rate * model.linear.weight.grad
  #   model.linear.bias -= learning_rate * model.linear.bias.grad
  optimizer.step()

  #zero gradient
  # model.linear.weight.grad.zero_()
  # model.linear.bias.grad.zero_()


  #print loss
  if epoch % 100 == 0:
    print(f'Epoch: {epoch}, Loss: {loss.item()}')


Epoch: 0, Loss: 0.5471685605474415
Epoch: 100, Loss: 0.10038536987100936
Epoch: 200, Loss: 0.08189594476401243
Epoch: 300, Loss: 0.0739752562391921
Epoch: 400, Loss: 0.06927668658560364
Epoch: 500, Loss: 0.0660703930723877
Epoch: 600, Loss: 0.0637084720898284
Epoch: 700, Loss: 0.06188320101936206
Epoch: 800, Loss: 0.060425085382600746
Epoch: 900, Loss: 0.05923099157135546
Epoch: 1000, Loss: 0.05823368062217064
Epoch: 1100, Loss: 0.05738710731134565
Epoch: 1200, Loss: 0.05665857066116905
Epoch: 1300, Loss: 0.05602417917398728
Epoch: 1400, Loss: 0.0554660695539843
Epoch: 1500, Loss: 0.054970622195616936
Epoch: 1600, Loss: 0.054527275699158086
Epoch: 1700, Loss: 0.05412771702083162
Epoch: 1800, Loss: 0.05376531513028795
Epoch: 1900, Loss: 0.05343471685497153
Epoch: 2000, Loss: 0.05313155326703479
Epoch: 2100, Loss: 0.052852222958427314
Epoch: 2200, Loss: 0.052593729781386066
Epoch: 2300, Loss: 0.05235355981621658
Epoch: 2400, Loss: 0.05212958702125081


In [69]:
  model.linear.bias

Parameter containing:
tensor([-0.3855], dtype=torch.float64, requires_grad=True)

In [70]:
#model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5452446937561035
